In [ ]:
#TODO Reformat this notebook. Move (most) of the explanation comments to their own markup cells.

#imports, global consts, inits
#The try-catch block bellow to avoid initialising stuff every run. Sometimes I run this entire workbook when changing input, but I don't need GEE/geemap to be
#reinitialised. This saves me a few seconds (a drop in the ocean, compared to how long computation of large rois take, but still...)
try:
    isInitialised

except: #Will always trigger if this is a new kernel (i.e. you restarted Jupyter).
    print ("Initialising Earth Engine API")
    isInitialised = True

    import ee
    import geemap

    #NOTE: If you haven't set a default project for earthengine to use via CLI, you'll need to provide ee.Initialize() with project name
    #see https://developers.google.com/earth-engine/guides/auth
    ee.Initialize()
    map = geemap.Map()

    sudanStateBorders = ee.FeatureCollection("projects/seamproj01/assets/SudanStateBorders")    #Shapefiles for Sudan administration borders, via OCHA HDX. This dataset is used for 
                                                                                                #state/admin division clipping (offline, on QGIS) of the cropland masks bellow. Mostly used
                                                                                                #when testing with geemap.

    testArea =  ee.FeatureCollection("projects/seamproj01/assets/test_area_v2") #A small block in the Gezira state east of the Blue Nile, a subset of geziraCropland

    khartoumCropland = ee.FeatureCollection("projects/seamproj01/assets/khartoum_cropmask_v4_1")    #From the Copernicus Moderate Dynamic Land Cover dataset. Extracted cropland pixels, then clipped
                                                                                                    #to Khartoum state. Polygons less than 0.3km2 in area are removed. Holes smaller than 0.5km2 are filled.
                                                                                                    #Subdivided using level 2 OCHA admin subdivions (with Karrari and Um Bada merged into one)
    geziraCropland = ee.FeatureCollection("projects/seamproj01/assets/gezira_cropmask_v4")  #Also based on Copernicus MDLC, cliped first with GlobCover dataset (via FAO LCLU). Then
                                                                                            #Seperated based on position relative to Blue Nile (east or west)

    sudanCropland = ee.FeatureCollection("projects/seamproj01/assets/sudan_cropmask_v1_2") #A polygon dataset that demarcates Sudan's crop lands, per type (irritaged, rainfed), per federal state.
    #This dataset were generated from Copernicus Moderate Dynamic Land Cover rasters combined with the ESA GlobCover (via FAO) dataset for irrigated land. The OCHA HDX Adminstrative Boudary data
    #was used for state (and substate localities) borders. The resulting data was furhter processed to eliminate small polygons (<1 km2), fill holes (<=0.5 km2) and mesh simplification (QGIS
    #grid method, 15 arsec threshold)

    testAreaSamples = ee.FeatureCollection("projects/seamproj01/assets/test_samples")   #50 samples covering testArea, with a string attribute NAME matching testArea's NAME attrib.
    
    #NOTE: If you are playing around with these datasets, note that their attributes are not all formatted the same way, so you will need to adjust e.g. subdivisionPropertyName for each.


In [ ]:
#User Input / Model Parameters

#===========================================================================================================================================
#Model Parameters
#===========================================================================================================================================

#roi is a feature collection containing target polygons for the region of analysis, each polygon marks a homogenious zone ("climatic divisions" in the original
#reasearch)
#projectName is a string with which the name of the output raster will be prefixed
roi : ee.FeatureCollection = testArea
projectName : str = "TestProj" 

#Trial projects
# roi = sudanCropland.filter(ee.Filter.eq("STATE", "Aj Jazirah"))
# projectName = "Gezira" 

# roi = sudanCropland.filter(ee.Filter.eq("STATE", "Aj Jazirah")).filter(ee.Filter.eq("TYPE", "Irrigated"))
# projectName = "GeziraIrrOnly" 

# roi = sudanCropland.filter(ee.Filter.eq("STATE", "Khartoum"))
# projectName = "Khartoum"

# roi = sudanCropland.filter(ee.Filter.eq("STATE", "Sennar"))
# projectName = "Sennar"

# roi = sudanCropland.filter(ee.Filter.eq("STATE", "Sennar")).filter(ee.Filter.inList(leftField = "COMPOUND_I", rightValue = ["Abu Hujar_Rainfed", "Ad Dali_Rainfed", "Sennar_Rainfed", "Sinja_Rainfed", "Sinja_Irrigated"]))
# projectName = "Sennar-W"

# roi = sudanCropland.filter(ee.Filter.eq("STATE", "Sennar")).filter(ee.Filter.inList(leftField = "COMPOUND_I", rightValue = ["At Dinder_Rainfed", "As Suki_Rainfed", "Sennar_Irrigated", "Sharg Sennar_Irrigated"]))
# projectName = "Sennar-E"

#This it the name of the property (or "attribute", in GIS-speak) that will be used to split the resulting fallow timeseries raster into "climatic divisions".
#NOTE: this is a required value. Even if your entire ROI is a single zone, you need to define some attribute here.
subdivisionPropertyName : str = "NAME"
# subdivisionPropertyName = "COMPOUND_I" #Supposed to be "COMPOUND_ID", but I forgot that shapefiles doesn't allow for feature name longer than 10 chars. Shapefiles must indeed die...

#exportSingleRaster controls whether the resulting fallow timeseries will be split to smaller rasters (one for each climatic zone, using subdivisionPropertyName) or 
#one, big raster for the entire ROI.
#NOTE: This would technically mean multiple smaller exports vs one big export. I'm not sure how this would affect the computations server side.
exportSingleRaster : bool = True

#This is the value for the z-score bellow which a pixel is considered fallow. Made a vairable here for caliberation purposes.
#temporalZScoreThresholdMax = -3 #Original value used by Wallace et al.
#temporalZScoreThresholdRange = -3 #Original value used by Wallace et al.
temporalZScoreThresholdMax : float = -1.055
temporalZScoreThresholdRange : float = -1.055

#For spatial anomaly analysis, the max of a climate division for a given season is multiplied by this value.
#spatialMedianMultiplierMax = 0.8 #Original value used by Wallace et al.
#spatialMedianMultiplierMax = 0.8 #Original value used by Wallace et al.
spatialMedianMultiplierMax : float = 0.855
spatialMedianMultiplierRange : float = 0.855

#time-series limits. All inclusive. 
#Months 1-12. User must make sure this range is covered by the selected dataset (MODIS / Sentinel)
yearStart : int = 2013
monthStart  : int = 1
yearEnd  : int = 2025
monthEnd  : int = 2

#targetMonths are the months comprising the season for analysis.
#WARNING! MUST BE 4 Values! Otherwise, the Temporal Anomaly Analysis component must be adjusted
#WARNING! ORDER OF MONTHS MUST BE CHRONOLOGICAL! If the season is inter-annual, start with the months in the first year, then the second year
#e.g. if the season starts on November, the list would be [11, 12, 1, 2]
targetMonths : list[int] = [7, 8, 9, 10] #"Summer" season in Sudan (technically Autumn. Don't ask...). This covers the growth periods of crops such as sorghum.
#targetMonths = [11, 12, 1, 2] #"Winter" season in Sudan (This one is indeed Winter). This covers growth periods of crops such as wheat

#The original implementation of this workbook allows for use of one of three datasets, "modis" for MODIS terra SR, "sentinel" for Sentinel-2 MSI SR, and "landsat" for
#LANDSAT 8 OLI Collection 2 Tier 2 SR. Set the string bellow to the desired dataset (obviously, mind your spelling)
dataset : str = "modis"
#NOTE: the Sentinel-2 L2A data used here miss some dates prior to dec 2018 for Gezira region. This is specific to GEE's version of 2A. Actual 2A on Copernicus Dataspace do cover this period.


#The spatial anomaly analysis has two a reduceRegion operations, which are very expensive, and can cause computation to fail (due to hitting memory/time limits)
#if the number of pixels in the rasters is too large, which is often the case with Sentinel data and large rois. reduceRegion() can be instructed to use a "tileScale"
#to workaround memory limitation (consult the docs for more info), this value is assigned using reduceRegionTileScaleOverride
#Value of the scale must be from 0.1 to 16.0. Default is 1.0.
#NOTE: tile scale is basically a compromise factor, it shifts the bottleneck between processing time and memory requirements. Lower scales are faster but require a
#lot of memory (i.e. you are bound to hit memory caps for large pixel count), while higher scales use less memory buy take longer to compute (so you are bound to hit
#processing timeout limits).
reduceRegionTileScaleOverride : float = 1.0
# reduceRegionTileScaleOverride = 6.0

#Simialiry, reduceReionNominalScaleOverride overrides the "scale" it computes with. If set to None, this script will use targetOutpuScale (which
#is 10m for Sentinel, 250m for MODIS). If set to an integer, it will use that instead. Mostly useless for MODIS (should be kept as None), but for
#Sentinel, greater values may be needed, depending on the size of the ROI. NOTE: this will affect the quality of the analysis.
reduceRegionNominalScaleOverride : int | None = None

#===========================================================================================================================================
#Auto Calibration settings
#===========================================================================================================================================
#if autoCalibrateParams is true, the model will be calibrated using calibrationDataset.
#calibrationAttribName is the attribute column name that contains integer marking whether land is fallow or not (0 = not fallow, 1 = fallow)
#calibrationYear is the year which the values in ISFALLOW represent
#NOTE: currently, auto calibration requires manual cell execution (because it involves waiting for GEE to finish an export job and Setting this to true requires
#manually executing this workbook to the cell with "#Auto-calibration first cell" heading), waiting for export to finish, then executing the remaining cells.
#NOTE: when auto calibrating, the final fallow timeseries will not be exported. Results for calibration (accuracy + parameters) will be output in the last
#relevant cell bellow.
autoCalibrateParams : bool = False
calibrationDataset : ee.FeatureCollection = testAreaSamples #Note: ensure that ALL the points ins this dataset fall within the your roi polygons (and subsequent rasters), else you'll get exceptions
calibrationAttribName : str = "ISFALLOW"
calibrationYear : int = 2019

#The calibration needs to export two images and store them. Exported to pathPrefix + "tempAn", and pathPrefix + "spStat"
#Adjust pathPrefix to point to your GEE project's assets (i.e. replace "seamproj01" with the name of your project)
pathPrefix = "projects/seamproj01/assets/"

#Differential Evolution parameters:
popSize: int = 30 #Population size. Minimum = 4 Recommended estimate is 10 * parameters. e.g. for four params: 10 * 4 = 40
crossOverProb: float = 0.9 #Cross Over Probability. Should be between 0.0 and 1.0. Recommended default = 0.9
diffWeight: float = 0.8 #Differential Weight. Between 0.0 and 2.0. Recommended default = 0.8
coupleParams: bool = True  #if True, zScore params shall be coupled, as well as spatialMedianMultipliers (i.e. 2 params instead of 4). Make sure to adjust 
                            #paramRanges bellow accordingely. Also note: coupling params means you probably could get away with a smaller popSize
paramRanges: list[list[float]] = [[-1.0, -4.0], [-1.0, -4.0], [0.5, 1.5], [0.5, 1.5]] #Allowble range for each parameter.
                                                                    #order = temporalZScoreThresholdMax, temporalZScoreThresholdRange, spatialMedianMultiplierMax,
                                                                    #and spatialMedianMultiplierRange
                                                                    #if coupling params, order = [zScoreThresholds, spatialMedianMultiliers]
diffEvSeed: int = 485138463513684685 #seed for the randomizer. Ensures reproducability. Setting to None makes each run different than preceding one
minIterations: int = 15 #Optimisation shall proceed for iteratios no less than this number
maxIterations: int = 30 #Optimisation shall break after this number of iterations, regardless of results
minChangeThreshold: float = 0.01 #minimum (percentage) change in objective function between iterations before stopping optimisation

In [ ]:
#Input ImageCollections and Dates Preprocessing

#===========================================================================================================================================
#Helper methods
#===========================================================================================================================================

#This function is especially usefull for sentinel data. To try and improve performance a bit, we'll limit the collection to only target months, using ee.Filter.calendarRange(start, end, field)
#problem is, targetMonths may no be in order (consider inter-annual years), so we can't just take first and last entry. can't take max or min either else it would defeat purpose (min/max
#of [11, 12, 1, 2] would result in twelve months).
def FilterCollectionForPeriod(  col : ee.ImageCollection,
                                startDate : str,
                                endDate : str,
                                targetMonths : list) -> ee.ImageCollection:
    periodStart = 0
    periodEnd = 0
    periods = []
    for i in range (1, len(targetMonths)):
        if (targetMonths[i] > targetMonths[i - 1]):
            periodEnd = i
        else:
            periods.append([targetMonths[periodStart], targetMonths[periodEnd]])
            periodStart = periodEnd = i

    periods.append([targetMonths[periodStart], targetMonths[periodEnd]])

    monthsFilter = ee.Filter.calendarRange(periods[0][0], periods[0][1], "month")
    for i in range (1, len(periods)):
        monthsFilter = ee.Filter.Or(monthsFilter, ee.Filter.calendarRange(periods[i][0], periods[i][1], "month"))

    return col.filterDate(startDate, endDate).filter(monthsFilter)

#Clipping image with high vertex-count polygons can take a LOT of time. So, we clip our rasters to the bounding box of these polygons, and suffer processing the extra
#pixels. From quick testing, this is still much faster than clipping to exact geometry. Computing NDVI TS for khartoum (v3) dataset (2019-2023 winter seasons) took only
#2 hours with the new clipping method. With the original, exact clipping, it wasn't finished even after 17hrs (canceled it at that point)
#Note that this doesn't affect the clipping optimisation in the spatial anomaly analysis component, neither the reduceRegion(s) used in it
def ClipCollectionToCollection(imageCol : ee.ImageCollection, featureCol : ee.FeatureCollection):
    clipGeometry = featureCol.map(lambda feature : ee.Feature(feature.geometry().convexHull())).geometry() 
    return imageCol.map(lambda image : image.clip(clipGeometry).copyProperties(image, image.propertyNames()))

def ProcessMODISCollection( col : ee.ImageCollection,
                            roi : ee.FeatureCollection,
                            startDate : str,
                            endDate : str,
                            targetMonths : list) -> ee.ImageCollection:
    
    #mappable functions (leaving them scoped inside the MODIS function because similarily named ones with different implementation exist for Sentinel as well)
    def MaskPoortQualityPixels(img : ee.Image) -> ee.Image:
        qaBand = img.select("State")
        #TODO add snow/ice masking
        mask = qaBand.bitwiseAnd(3).eq(0) #pixel is clear from cloud (bits 0 and 1)
        mask = mask.And(qaBand.bitwiseAnd(4).eq(0)) #not cloud shadow (bit 2)
        mask = mask.And(qaBand.bitwiseAnd(768).eq(0)) #no cirrus (bits 8 and 9)
        return img.updateMask(mask)
    
    output = FilterCollectionForPeriod(col, startDate, endDate, targetMonths)
    output = output.filterBounds(roi.geometry())
    #output = output.map(lambda image : image.clip(roi).copyProperties(image, image.propertyNames()))
    output = ClipCollectionToCollection(output, roi)
    output = output.map(MaskPoortQualityPixels)
    output = output.map(lambda image : image.normalizedDifference(["sur_refl_b02", "sur_refl_b01"]).rename("NDVI").copyProperties(image, image.propertyNames()))
    return output

def ProcessSentinelCollection(  col : ee.ImageCollection,
                                roi : ee.FeatureCollection,
                                startDate : str,
                                endDate : str,
                                targetMonths : list) -> ee.ImageCollection:
    
    def MaskPoortQualityPixels(img : ee.Image) -> ee.Image:
        qaBand = img.select("SCL")
        #Unlike MODIS, the quality band used here, "SCL," contains a single int meant to be interpreted as a single int.
        #https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/scene-classification/
        
        mask = qaBand.eq(3) #cloud shadows
        mask = mask.And(qaBand.eq(6)) #water
        mask = mask.And(qaBand.eq(8)) #medium probability clouds
        mask = mask.And(qaBand.eq(9)) #high probability clouds
        mask = mask.And(qaBand.eq(10)) #thin cirrus
        mask = mask.And(qaBand.eq(11)) #snow/ice

        mask = mask.neq(1) #flip so it masks out the above
        
        return img.updateMask(mask)
    
    #filter then process the output collection
    output = FilterCollectionForPeriod(col, startDate, endDate, targetMonths)
    output = output.filterBounds(roi.geometry())
    output = output.select("B4", "B8", "SCL") #grasping at straws trying to make this thing run faster... TODO experiment to see if this actually has an effect
    #output = output.map(lambda image : image.clip(roi).copyProperties(image, image.propertyNames()))
    output = ClipCollectionToCollection(output, roi)
    output = output.map(MaskPoortQualityPixels)
    output = output.map(lambda image : image.normalizedDifference(["B8", "B4"]).rename("NDVI").copyProperties(image, image.propertyNames()))
    return output

#TODO The MaskPoorQualityPixels() method for landsat is very similar to modis. Could use some DRYing.
#TODO the bodies of these three functions is highly similar. DRY it.
def ProcessLandsatCollection    (col : ee.ImageCollection,
                                roi : ee.FeatureCollection,
                                startDate : str,
                                endDate : str,
                                targetMonths : list) -> ee.ImageCollection:
    
    def MaskPoortQualityPixels(img : ee.Image) -> ee.Image:
        qaBand = img.select("QA_PIXEL")
        
        mask = qaBand.bitwiseAnd(4).eq(0) #no cirrus detected (bit 2)
        # mask = mask.And(qaBand.bitwiseAnd(64).eq(0)) #pixel set to clear (no clouds) (bit 6) #is this for the entire image, not the pixel?
        mask = mask.And(qaBand.bitwiseAnd(8).eq(0)) #no cloud (bit 3)
        mask = mask.And(qaBand.bitwiseAnd(16).eq(0)) #no cloud shadow (bit 4)
        mask = mask.And(qaBand.bitwiseAnd(128).eq(0)) #no water (bit 7)
        return img.updateMask(mask)
    
    #filter then process the output collection
    output = FilterCollectionForPeriod(col, startDate, endDate, targetMonths)
    output = output.filterBounds(roi.geometry())
    output = output.select("SR_B4", "SR_B5", "QA_PIXEL")
    output = ClipCollectionToCollection(output, roi)
    output = output.map(MaskPoortQualityPixels)
    output = output.map(lambda image : image.multiply(0.0000275).subtract(0.2).normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI").copyProperties(image, image.propertyNames()))#.set({"_year" : ee.Date.format(image.get("system:time_start"), "YYYY")}))
    return output
    

#===========================================================================================================================================
#Preprocessing
#===========================================================================================================================================

dateStart = f"{yearStart}-{monthStart}-1"
dateEnd = f"{yearEnd}-{monthEnd + 1}-1" if monthEnd < 12 else  f"{yearEnd + 1}-1-1"

processedCol = None

if dataset == "sentinel":
    processedCol = ProcessSentinelCollection( ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED"), roi, dateStart, dateEnd, targetMonths)
    targetOutputScale = 10
elif dataset == "modis":
    processedCol = ProcessMODISCollection(ee.ImageCollection("MODIS/061/MOD09Q1"), roi, dateStart, dateEnd, targetMonths)
    targetOutputScale = 250
elif dataset == "landsat":
    processedCol = ProcessLandsatCollection(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"), roi, dateStart, dateEnd, targetMonths)
    targetOutputScale = 30

ndviTS = processedCol

In [ ]:
#In this block, we generte the timeseries (monthly max NDVI and NDVI range) and the statistics for the "pure crop" signal.
#Outputs of this block are two ImageCollections: "timeseries," and "pureCropNDVI"
#Note: "year" here is used to refer to the year of the start of the season, no the calendar year of the month. This is important for inter-annual seasons (e.g. Nov through
#Feb). 
    #timseries contains images for each season.
        #Each Image has number of bands equal to twice the number of targetMonths (2x4 = 8).
        #Each Image has a property "year" for the season's year.
        #Each band is named "month_x_max" or "month_x_range", where x is the month's number; "max" and "range" denote whether it encodes the maximum monthly ndvi or ndvi monthly range.
    #pureCropNDVI contains images for each month in the targetMonths (total = 4)
        #each image has 4 bands: "month_x_max_mean" or "month_x_max_stdDev" (and similarily for range).
        #each image has a property "month" with month's number

timeSeries = ee.List([])

lastCompleteSeason = yearStart

for year in range(yearStart, yearEnd + 1):
    
    yearTS = ee.List([])
    isCompleteSeason = True
    
    lastMonth = 0
    calendarYear = year
    lastCompleteSeason = year

    for month in targetMonths:
        if (month < lastMonth):
            calendarYear += 1
        lastMonth = month
        
        if ((calendarYear * 100) + month > (yearEnd * 100) + monthEnd):
            isCompleteSeason = False
            break
        
        
        subPeriodStart = f"{calendarYear}-{month}-1"
        subPeriodEnd = f"{calendarYear}-{month + 1}-1" if month < 12 else  f"{calendarYear + 1}-1-1"
        subCol = ndviTS.filterDate(subPeriodStart, subPeriodEnd)
        
        minMaxNDVI = subCol.reduce(ee.Reducer.minMax())

        monthMax = minMaxNDVI.select("NDVI_max").rename("max")
        monthRange = minMaxNDVI.select("NDVI_max").subtract(minMaxNDVI.select("NDVI_min")).rename("range")
        yearTS = yearTS.add(ee.Image([monthMax, monthRange]).set({"system:index" : f"month_{month}"}))
    
    if(not isCompleteSeason):
        lastCompleteSeason += -1
        print (f"WARNING! Reached an incomplete season at year {lastCompleteSeason + 1}, timeseries will end at {lastCompleteSeason}")
        break
    
    yearTS = ee.ImageCollection(yearTS).toBands().set({"year" : year, "system:index" : ee.String("year_").cat(str(year))})
    timeSeries = timeSeries.add(yearTS)

timeSeries = ee.ImageCollection(timeSeries)

##uncomment this part to export the ndviTS. Not very recommended for Sentinel 2 data for large areas (produces rasters between 500~900MB for Gezira dataset)...
##to reduce output size, the data is scaled by 100 then rounded off, and stored as an unsigned, 8bit int which range 0 to 255, but because ndvi ts ranges from 0 
##to 1 (zero because we masked out water), the resulting raster would range from 0 to 100. Also note the reduced precision this introduces (shouldn't matter
##much anyway)
##TODO Extra precision could be obtained by using the full 0~255 range by multiplying Ndvi with 255 then truncating decimals. This would allow for the storage
##of down to 0.004 difference (more precise than 0.01 of range 0~100) Retrival is carried out by subdividing by 255.
##TODO adjust code so bands would have name of year.
##TODO seperate output images for this the same as you do fallowTS bellow.
##TODO export max and range timeseries seperately
# task = ee.batch.Export.image.toDrive(
#     image = timeSeries.select([f"month_{targetMonths[0]}_max", f"month_{targetMonths[1]}_max", f"month_{targetMonths[2]}_max", f"month_{targetMonths[3]}_max"]).toBands().clip(roi.geometry()).multiply(ee.Image(ee.Number(100))).toUint8(),
#     description = f"{projectName}_maxNDVI_TS_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[1]}-{targetMonths[2]}-{targetMonths[3]}",
#     # image = timeSeries.select([f"month_{targetMonths[0]}_range", f"month_{targetMonths[1]}_range", f"month_{targetMonths[2]}_range", f"month_{targetMonths[3]}_range"]).toBands().clip(roi.geometry()).multiply(ee.Image(ee.Number(100))).toUint8(),
#     # description = f"{projectName}_rangeNDVI_TS_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[1]}-{targetMonths[2]}-{targetMonths[3]}",
#     #folder='ee_export',
#     region = roi.geometry(),
#     scale = targetOutputScale,
#     crs = 'EPSG:4326',
#     maxPixels = 500000000,
#     fileFormat = 'GeoTIFF',
#     formatOptions = {
#         'noData': -9999
# })
# task.start()


pureCropNDVI = ee.List([])

for month in targetMonths:
    prefix = f"month_{month}_"
    thisMonthTS = timeSeries.select([prefix + "max", prefix + "range"])

    monthMedian = thisMonthTS.reduce(ee.Reducer.median()).rename([prefix + "max", prefix + "range"]).set({"month" : month})
    
    meanStdReducer = ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs = True)

    monthPureCropNDVI = thisMonthTS.map(lambda img : img.updateMask(img.gte(monthMedian)))
    monthPureCropNDVI = monthPureCropNDVI.reduce(meanStdReducer).set("month", month)

    pureCropNDVI = pureCropNDVI.add(monthPureCropNDVI)

pureCropNDVI = ee.ImageCollection(pureCropNDVI)

In [ ]:
#This block computes the temporal anomalies (z-scores) for each season. The output is an ImageCollection called temporalAnomalies, containing Images representing each season.
    #Each Image has number of bands equal to twice the number of targetMonths (2x4 = 8).
    #Each Image has a property "year" for the season's year.
    #Each band is named "month_x_max" or "month_x_range", respectively for the z-scores for max NDVI and NDVI range, for each target month.

def ComputeTemporalAnomalies(image : ee.Image) -> ee.Image:
    seasonAnomalies = ee.List([])

    for month in targetMonths:
        prefix = f"month_{month}_"
        pureCropSignal = pureCropNDVI.filter(ee.Filter.eq("month", month)).first()
        
        taMax = image.select(prefix + "max").subtract(pureCropSignal.select(prefix + "max_mean")).divide(pureCropSignal.select(prefix + "max_stdDev")).rename("max")
        taRange = image.select(prefix + "range").subtract(pureCropSignal.select(prefix + "range_mean")).divide(pureCropSignal.select(prefix + "range_stdDev")).rename("range")

        ta = ee.Image([taMax, taRange]).set({"system:index" : f"month_{month}"})
        seasonAnomalies = seasonAnomalies.add(ta)
    
    return ee.ImageCollection(seasonAnomalies).toBands().set({"year" : image.get("year")})
    
temporalAnomalies = timeSeries.map(ComputeTemporalAnomalies)

##uncomment this part to export the anomaly series.
##similarily to ndvi timeseries export, the data is scaled by 100 the rounded off, and stored as a 16bit int with range -32,768 to 32,767
##since z score shouldn't go far from zero (negative or positive single digits, typically), this range should be more than adequte.
##Note: gee refuses to set image index (its name) inside map(). So the output image will be prefixed with integers starting from 0, instead of year number.
##TODO export max and range series seperately
# task = ee.batch.Export.image.toDrive(
#     image = temporalAnomalies.toBands().clip(roi.geometry()).multiply(ee.Image(ee.Number(100))).toInt16(),
#     description = f"{projectName}_temporalAnomalies_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[1]}-{targetMonths[2]}-{targetMonths[3]}",
#     #folder='ee_export',
#     region = roi.geometry(),
#     scale = targetOutputScale, #WARNING! The zscore data isn't very compressible, even when reduced to int16. Using large scales (less than 30m) with large areas (gezira example) results in rasters 11GB in size
#     crs = 'EPSG:4326',
#     maxPixels = 500000000,
#     fileFormat = 'GeoTIFF',
#     formatOptions = {
#         'noData': -9999
# })
# task.start()


In [ ]:
#Temporal analysis component.

#mapable function, mapped over temporalAnomalies collection
def TemporalAnomalyAnalysis(image : ee.Image) -> ee.Image: #to be mapped over temporalAnomalies collection
    isFallow_1 = image.select(f"month_{targetMonths[0]}_max").lt(temporalZScoreThresholdMax).And(image.select(f"month_{targetMonths[1]}_max").lt(temporalZScoreThresholdMax)).And(image.select(f"month_{targetMonths[2]}_max").lt(temporalZScoreThresholdMax))
    isFallow_1 = isFallow_1.Or(image.select(f"month_{targetMonths[1]}_max").lt(temporalZScoreThresholdMax).And(image.select(f"month_{targetMonths[2]}_max").lt(temporalZScoreThresholdMax)).And(image.select(f"month_{targetMonths[3]}_max").lt(temporalZScoreThresholdMax)))
    isFallow_1 = isFallow_1.rename("max")

    isFallow_2 = image.select(f"month_{targetMonths[0]}_range").lt(temporalZScoreThresholdRange).And(image.select(f"month_{targetMonths[1]}_range").lt(temporalZScoreThresholdRange)).And(image.select(f"month_{targetMonths[2]}_range").lt(temporalZScoreThresholdRange))
    isFallow_2 = isFallow_2.Or(image.select(f"month_{targetMonths[1]}_range").lt(temporalZScoreThresholdRange).And(image.select(f"month_{targetMonths[2]}_range").lt(temporalZScoreThresholdRange)).And(image.select(f"month_{targetMonths[3]}_range").lt(temporalZScoreThresholdRange)))
    isFallow_2 = isFallow_2.rename("range")

    return ee.Image([isFallow_1, isFallow_2]).set({"year" : image.get("year")})

#isFallow_TA the two questions for the temporal anomalies
isFallow_TA = temporalAnomalies.map(TemporalAnomalyAnalysis)

In [ ]:
#Spatial analysis component.

#mapable function, mapped over timeseries collection to generate input for spatial anomally analysis
def SpatialStatistics(image : ee.Image) -> ee.Image:
    maxOfMax = ee.List([])
    maxOfRange = ee.List([])
    
    spatialMedianMax = ee.FeatureCollection([])
    spatialMedianRange = ee.FeatureCollection([])

    for month in targetMonths:
        name = f"month_{month}"
        _maxOfMax : ee.Image = image.select(name + "_max").rename("max")
        _maxOfRange : ee.Image = image.select(name + "_range").rename("range")

        maxOfMax = maxOfMax.add(_maxOfMax)
        maxOfRange = maxOfRange.add(_maxOfRange)
        
        zoneMaxMedian = _maxOfMax.reduceRegions(collection = roi,
                                                reducer = ee.Reducer.median(),
                                                scale = targetOutputScale if reduceRegionNominalScaleOverride is None else reduceRegionNominalScaleOverride,
                                                crs = "EPSG:4326",
                                                tileScale = reduceRegionTileScaleOverride)

        zoneRangeMedian =  _maxOfRange.reduceRegions(   collection = roi,
                                                        reducer = ee.Reducer.median(),
                                                        scale = targetOutputScale if reduceRegionNominalScaleOverride is None else reduceRegionNominalScaleOverride,
                                                        crs = "EPSG:4326",
                                                        tileScale = reduceRegionTileScaleOverride)
        
        spatialMedianMax = spatialMedianMax.merge(zoneMaxMedian)
        spatialMedianRange = spatialMedianRange.merge(zoneRangeMedian)
        
    maxOfMax = ee.ImageCollection(maxOfMax).reduce(ee.Reducer.max(), parallelScale = 4).rename("maxOfMax")
    maxOfRange = ee.ImageCollection(maxOfRange).reduce(ee.Reducer.max(), parallelScale = 4).rename("maxOfRange")

    zonesMedians = roi.map(lambda feature : feature.set({"medianOfMax" : ee.Number(spatialMedianMax.filter(ee.Filter.eq(subdivisionPropertyName, feature.get(subdivisionPropertyName))).aggregate_array("median").reduce(ee.Reducer.max()))}))
    zonesMedians = zonesMedians.map(lambda feature : feature.set({"medianOfRange" : ee.Number(spatialMedianRange.filter(ee.Filter.eq(subdivisionPropertyName, feature.get(subdivisionPropertyName))).aggregate_array("median").reduce(ee.Reducer.max()))}))
    
    #mappable function, mapped over a list of features to generate a list of fixed value rasters (with two bands) clipped to the features.
    def RasterFromVectorStats(feature : ee.Feature) -> ee.Image:
        feature = ee.Feature(feature)
        medianOfMax = ee.Image(ee.Number(feature.get("medianOfMax"))).clip(feature.geometry()).toFloat()
        medianOfRange = ee.Image(ee.Number(feature.get("medianOfRange"))).clip(feature.geometry()).toFloat()
        return ee.Image([medianOfMax, medianOfRange]).rename(["medianOfMax", "medianOfRange"])
    
    #Generate a single image (with two bands) from the zonesMedians
    spatialMedians = ee.ImageCollection(zonesMedians.toList(zonesMedians.size()).map(RasterFromVectorStats)).mosaic()

    return ee.Image([maxOfMax, maxOfRange,
                     ee.Image(spatialMedians.select("medianOfMax")), 
                     ee.Image(spatialMedians.select("medianOfRange"))]).set({"year" : image.get("year")})

#mapable function, mapped over spatial statistics to generate the isFallow rasters based on spatial anomalies
def SpatialAnomalyAnalysis(image : ee.Image) -> ee.Image:
    maxOfMax = image.select("maxOfMax")
    maxOfRange = image.select("maxOfRange")
    spatialMedianMax = image.select("medianOfMax")
    spatialMedianRange = image.select("medianOfRange")

    isFallow_3 = maxOfMax.lt(spatialMedianMax.multiply(ee.Number(spatialMedianMultiplierMax))).rename("max")
    isFallow_4 = maxOfRange.lt(spatialMedianRange.multiply(ee.Number(spatialMedianMultiplierRange))).rename("range")

    return ee.Image([isFallow_3, isFallow_4]).set({"year" : image.get("year")})


#isFallow_SA the two questions for the spatial anomalies
spatialStats = timeSeries.map(SpatialStatistics)
isFallow_SA = spatialStats.map(SpatialAnomalyAnalysis)

In [ ]:
#Final analysis.
#For a pixel to qualify as fallow, it has to be classified as such in at least two of either the temporal or spatial checks.

def ComputeFallowTS(isFallow_TA, isFallow_SA):
    isFallowComponents = ee.List([])
    for year in range (yearStart, yearEnd + 1):
        if ((year * 100) + targetMonths[0] > (yearEnd * 100) + monthEnd):
                break
                
        yearComponents = ee.List([]).add(isFallow_TA.filter(ee.Filter.eq("year", year)).first().set({"system:index" : f"{year}_TA"}))
        yearComponents = yearComponents.add(isFallow_SA.filter(ee.Filter.eq("year", year)).first().set({"system:index" : f"{year}_SA"}))

        yearComponents = ee.ImageCollection(yearComponents).toBands().set({"year" : str(year)})
        isFallowComponents = isFallowComponents.add(yearComponents)

    isFallowComponents = ee.ImageCollection(isFallowComponents)
    
    return isFallowComponents.map(lambda img : img.reduce(ee.Reducer.sum()).gte(ee.Number(2)).rename(ee.String(img.get("year")))).toBands()


fallowTS = ComputeFallowTS(isFallow_TA, isFallow_SA)

In [ ]:
#Auto-calibration first cell
#Cache temporatlAnomalies and spatialStats (which are expensive to compute) as an asset to be used for optimisation

#NOTE: the "task.start()" calls are commented out for testing, since this entire workbook may be recomputed multiple times during optimisation (either due to
#optimisation param changes, debugging, etc). Uncomment those callse and run this cell ONCE. Wait for the export to finish (check your GEE or Google Cloud account's
#task manager), then comment those lines out.
#You will need to rerun this for every roi change.
#You will need to rerun this if you changed the target months, or if you expanded your year ranges (if new range is within than old one, you don't need to recompute)

#TODO gee complains about overwriting assets. Is there a flag in the export to force it? If not, think of an elegant way to handle this.

#To make life a little bit easier, clip the rasters above to a small area around the sampling points, since we don't need the whole region for calibration,
#which is what we are doing if we are in this block

clipper = testAreaSamples.map(lambda point : point.buffer(targetOutputScale)) #ideally, we would clip only to the sampled pixels, but getting their exact positions may be time consuming
#buffer on a point returns a circle, let's convert it to a rectangle. Not necessary, but I like rects better for these kinda things :) Using the circles bounds simplifis this.
clipper = clipper.map(lambda circle : ee.Geometry.Rectangle(ee.Array.cat(circle.geometry().bounds().coordinates(),1).slice(0, 0, 3, 2).reshape([-1]).toList()))

if autoCalibrateParams:
    task = ee.batch.Export.image.toAsset(temporalAnomalies.toBands().clip(clipper.geometry()), 
                                        "temporalAnomalies",  
                                        pathPrefix + "tempAn", 
                                        scale = targetOutputScale, 
                                        maxPixels= 3e8)
    # task.start() #NOTE uncomment this to generate the temporal anomaly raster for calibration, then comment out.

    task = ee.batch.Export.image.toAsset(spatialStats.toBands().clip(clipper.geometry()), 
                                        "spatialStats",  
                                        pathPrefix + "spStat", 
                                        scale = targetOutputScale, 
                                        maxPixels= 3e8)

    # task.start() #NOTE uncomment this to generate the spatial statistics raster for calibration, then comment out.

In [ ]:
#Recreate temporaAnomalies and spatialStats from cached rasters

if autoCalibrateParams:
    tempAnImg = ee.Image(pathPrefix + "tempAn")
    spStatImg = ee.Image(pathPrefix + "spStat")

    temporalAnomalies = ee.List([])
    spatialStats = ee.List([])

    spStatsNames = ["maxOfMax", "maxOfRange", "medianOfMax", "medianOfRange"]
    taBandNames = [] #the process bellow breaks the band naming (prefexes an integer), generate this list for easier renaming bellow.
    for m in targetMonths:
        taBandNames.append(f"month_{m}_max")
        taBandNames.append(f"month_{m}_range")

    yearsList = [int(i[5:9]) for i in tempAnImg.bandNames().getInfo()]  #workaround for an issue introduced by inter-annual seasons.

    for year in range(yearStart, yearEnd + 1):
        if year not in yearsList:  #workaround for an issue introduced by inter-annual seasons.
            continue

        yearImgTA = ee.List([])
        #temporal anomalies are by month, so we loop over targetMonths
        for month in targetMonths:
            bandNameMax = f"year_{year}_month_{month}_max"
            bandNameRange = f"year_{year}_month_{month}_range"
            
            yearImgTA = yearImgTA.add(tempAnImg.select(bandNameMax).rename([f"month_{month}_max"]))
            yearImgTA = yearImgTA.add(tempAnImg.select(bandNameRange).rename([f"month_{month}_range"]))

        #spatialStats are for an entire year, not by month.
        yearImgSS = ee.ImageCollection([spStatImg.select(f"year_{year}_{statName}") for statName in spStatsNames]).toBands().rename(spStatsNames).set({"year" : year})

        temporalAnomalies = temporalAnomalies.add(ee.ImageCollection(yearImgTA).toBands().set({"year" : year}).rename(taBandNames))
        spatialStats = spatialStats.add(yearImgSS)

    temporalAnomalies = ee.ImageCollection(temporalAnomalies)
    spatialStats = ee.ImageCollection(spatialStats)

    # #test
    # print ("for tempAn")
    # print (temporalAnomalies.first().bandNames().getInfo())
    # print (temporalAnomalies.aggregate_array("year").getInfo())
    # print ("for spStat")
    # print (spatialStats.first().bandNames().getInfo())
    # print (spatialStats.aggregate_array("year").getInfo())

    # map.addLayer(clipper, name="clipper")
    # map.addLayer(tempAnImg, name="tempAn")
    # map.addLayer(spStatImg, name="spStat")
    # map.addLayer(testAreaSamples, name="samples")

    # map.centerObject(testAreaSamples.geometry())
    # map

In [ ]:
#Autocaliberation using Differential Evolution
#https://en.wikipedia.org/wiki/Differential_evolution

if autoCalibrateParams:
    from random import seed as Seed, random as Random, randrange as RandIntRange

    #Should usually be called before RecomputeFallowTS(), else the latter will use old parameters.
    def UpdateParams(parameters):
        global temporalZScoreThresholdMax
        global temporalZScoreThresholdRange
        global spatialMedianMultiplierMax
        global spatialMedianMultiplierRange

        if coupleParams:
            temporalZScoreThresholdMax = temporalZScoreThresholdRange = parameters[0]
            spatialMedianMultiplierMax = spatialMedianMultiplierRange =  parameters[1]
        else:
            temporalZScoreThresholdMax = parameters[0]
            temporalZScoreThresholdRange = parameters[1]
            spatialMedianMultiplierMax = parameters[2]
            spatialMedianMultiplierRange = parameters[3]

    def RecomputeFallowTS():
        return ComputeFallowTS(temporalAnomalies.map(TemporalAnomalyAnalysis),
                            spatialStats.map(SpatialAnomalyAnalysis))

    #assumes fallowTS is up to date
    def SampleIsFallowAtValidationPoints(fallowTS) -> ee.FeatureCollection:
        isFallowRaster = fallowTS.select(f"..{calibrationYear}") #output of fallowTS generation prepends an int from 0 onwards then an underscore.
        bandName = isFallowRaster.bandNames().get(0)

        #mappable function
        def Sample(point):
            value = ee.Number(isFallowRaster.sample(region = point.geometry(), scale = targetOutputScale).first().get(bandName))
            return point.set({"estimate" : value})

        return testAreaSamples.map(Sample)
        

    #assumes fallowTS is up to date
    def ObjectiveFunction(fallowTS):
        data = SampleIsFallowAtValidationPoints(fallowTS)

        #mappable function, to help with confusion matrix generation
        def AddCompositeColumn(point):
            return point.set({"compositeScore" : ee.Number((ee.Number(2).multiply(point.get("estimate")).add(point.get(calibrationAttribName))))})

        adjustedSamples = ee.Dictionary(data.map(AddCompositeColumn).aggregate_array("compositeScore").reduce(ee.Reducer.frequencyHistogram()))
        confArray = ee.Array([[adjustedSamples.get("3.0", 0.0), adjustedSamples.get("1.0", 0.0)], [adjustedSamples.get("2.0", 0.0), adjustedSamples.get("0.0", 0.0)]], ee.PixelType.float())
        confMatrix = ee.ConfusionMatrix(confArray.long())
        
        return confMatrix.accuracy().getInfo()

    def StopOptimisation(currentIteration, currentScore, lastScore) -> bool:
        if currentIteration < minIterations:
            return False

        if currentIteration >= maxIterations:
            print(f"Stopping after reaching max iterations")
            return True

        errorMetricChange = abs((lastScore - currentScore) / lastScore)
        if errorMetricChange <= minChangeThreshold:
            print(f"Stopping for stagnating improvements. Change percentage = {errorMetricChange}")
            return True
        
        return False

    def RandomParams(ranges: list[list]) -> list:
        params = []
        for range in ranges:
            params.append(range[0] + Random() * (range[1] - range[0]))
        return params

    def RandomAgents(agents: list[list], excludeID: int, count = 3) -> list[list]:
        picksIDs = []
        
        while len(picksIDs) < count:
            i = RandIntRange(0, len(agents))
            if i not in picksIDs and i != excludeID:
                picksIDs.append(i)

        return [agents[i] for i in picksIDs]

    def Clamp(value: float, range: list[float, float]) -> float:
        return max(min(value, max(range[0], range[1])), min(range[0], range[1]))

    #init population and run vars
    agents = [] #TODO consider switching agents and scores lists to a single priority queue (heapq). 
    scores = [] #same index as agents
    bestScore = [0.0, []] #[score, [params giving this score]]
    currentScore = 0.0
    iteration = 0

    Seed(diffEvSeed)

    #check to avoid logical errors in the input paramRanges
    if coupleParams and len(paramRanges) > 2:
        print (f"Warning! paramRanges supplied greater than 2.\nAre you using ranges for decoupled params?\nAttempting to use first and third ranges")
        paramRanges = [paramRanges[0], paramRanges[2]]
        print (f"Using ranges {paramRanges}")

    #while the theory says agents should be randomized, let's set the first two to something familiar (the defaults from the original research, and midpoints of the ranges)
    agents += [[-3.0, -3.0, 0.8, 0.8] if not coupleParams else [-3.0, 0.8], #Wallace et al
            [(paramRanges[i][0] + paramRanges[i][1]) / 2.0 for i in range(0, len(paramRanges))]]
    for initalAgent in agents: #compute their scores
        UpdateParams(initalAgent)
        scores.append(ObjectiveFunction(RecomputeFallowTS()))

    #generate the remaining agents (randomly)
    for i in range (0, popSize - len(agents)): #the subtraction to avoid bugs in case I changed the initial values above.
        randAgent = RandomParams(paramRanges)
        while randAgent in agents: #Should be a practical impossibility to happen, but still...
            randAgent = RandomParams(paramRanges)
        agents.append(randAgent)
        UpdateParams(randAgent)
        scores.append(ObjectiveFunction(RecomputeFallowTS()))

    #init bestScore from our randomely generated agents
    for i in range(0,popSize):
        if scores[i] > bestScore[0]:
            bestScore = [scores[i], agents[i]]

    print (f"Agents = {len(agents)}")
    for i in range (0, popSize): print(f"{scores[i]} -- {agents[i]}")

    print ("==================================\n\t\t\t\tStarting optimisation loop\n==================================")
    while not StopOptimisation(iteration, currentScore, bestScore[0]):
        print (f"------------------------------\nIteration {iteration} -- params: {bestScore[1]} -- score current: {bestScore[0]} last: {currentScore}")
        currentScore = bestScore[0]

        for i in range (0, popSize):
            assistVectors = RandomAgents(agents, i, 3)

            testedAgent = list(agents[i])
            rIndex = RandIntRange(0, len(paramRanges))
            weights = RandomParams([[0.0, 1.0]] * len(paramRanges))
            for j in range (0, len(paramRanges)):
                if weights[j] < crossOverProb or j == rIndex:
                    testedAgent[j] = Clamp(assistVectors[0][j] + diffWeight * (assistVectors[1][j] - assistVectors[2][j]), paramRanges[j])

            UpdateParams(testedAgent)
            testedScore = ObjectiveFunction(RecomputeFallowTS())

            if (testedScore > scores[i]):
                print (f"\t\tAgent {i} improved from {scores[i]} to {testedScore} - Now: {testedAgent}")
                agents[i] = testedAgent
                scores[i] = testedScore
            else:
                print (f"\t\tAgent {i} no improvement ({scores[i]})")
            
            if(testedScore > bestScore[0]):
                bestScore = [testedScore, testedAgent]

        print(f"Best scorer: {bestScore[1]} -- {bestScore[0]}")
        iteration += 1
        pass

    print (f"=================\n Finished after {iteration} iterations")
    print (f"{bestScore[0]} -- {bestScore[1]}")

In [ ]:
#Result export (when not calibrating)

if not autoCalibrateParams:
    noDataValue = -9999

    #split raster into multiple ones based on zone, then export
    if exportSingleRaster: 
        exportName = f"{projectName}_allZones_{dataset}_fallow-ts_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[1]}-{targetMonths[2]}-{targetMonths[3]}"
        print (f"exporting file: {exportName}")
        task = ee.batch.Export.image.toDrive(image = fallowTS.clip(roi.geometry()),
                                                 description = exportName,
                                                 region = roi.geometry(),
                                                 scale = targetOutputScale,
                                                 crs = 'EPSG:4326',
                                                 maxPixels = 300000000,
                                                 fileFormat = 'GeoTIFF',
                                                 formatOptions = {'noData': noDataValue})

        task.start()
    else:
        zones = roi.toList(roi.size())
        zonesSize = zones.size().getInfo()

        for i in range (0, zonesSize):
            targetZone = ee.Feature(zones.get(i))
            
            exportName = f"{projectName}_{targetZone.get(subdivisionPropertyName).getInfo()}_{dataset}_fallow-ts_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[1]}-{targetMonths[2]}-{targetMonths[3]}"
            print (f"{i} - exporting file: {exportName}")
            
            task = ee.batch.Export.image.toDrive(image = fallowTS.clip(targetZone.geometry()),
                                                 description = exportName,
                                                 region = targetZone.geometry(),
                                                 scale = targetOutputScale,
                                                 crs = 'EPSG:4326',
                                                 maxPixels = 300000000,
                                                 fileFormat = 'GeoTIFF',
                                                 formatOptions = {'noData': noDataValue},)
                                                 #priority = i) can only be set for "commercial projects"

            task.start()